In [1]:
#libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#variables
matriz = []

In [2]:
#input variables
NumUsers = int(input("Numero de usuarios: "))
proConnect =  float(input("Probabilidad que se conecte un usuario: "))
proDisconnect =  float(input("Probabilidad que se desconecte un usuario: "))

In [ ]:
#functions

In [ ]:
#code

In [4]:
#graphics
print(NumUsers , " ** ",proConnect, " ** ",proDisconnect)

5  **  0.6  **  0.2
